In [173]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import logging
from helper import getModelDict
delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

c = 3e8

FORMAT = '%(levelname)s: %(message)s at %(asctime)s'
logging.basicConfig(format=FORMAT,datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()

In [174]:
defaultPars = {'figure.figsize': (5, 4),
               'lines.markersize' : 4,
               'axes.titlesize' : 13,
               'font.size' : 13,
               'axes.labelsize' : 16,
               'xtick.labelsize' : 13,
               'ytick.labelsize' : 13,
               'legend.fontsize' : 10,
               "text.usetex": True,
               "font.family": "sans-serif",
               "font.sans-serif": ["Helvetica"],
               'font.family':'Times New Roman', 
               'font.serif':'Times New Roman',
               "savefig.dpi" : 300,
               'contour.linewidth' : 2.0,
               'lines.linewidth' : 2.0,
               'axes.grid' : True,
               'grid.linewidth' : 0.5,
               'grid.color' : 'lightgray',
               'axes.axisbelow' : True
               }
plt.rcParams.update(defaultPars)

### Delphes (ROOT) file

In [175]:
# file = './pp2chi0chi0J_scan/Events/run_01/ddmH_mS_500_m1_244_dm_90_delphes_events_nosmear_all.root'
file = './pp2chi0chi0J_scan/Events/run_01/ddmH_mS_500_m1_244_dm_90_delphes_events_all.root'
c_light = 2.99792458e8

### Check single event

### Check L1 Calorimeter

In [176]:
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)

print('\nOnTime Calorimeter:')

for tower in tree.L1TowerOnTime:
    pdgs = [p.PID for p in tower.Particles]
    # if 321 not in pdgs:
        # continue
    # energy = np.array([p.E for p in tower.Particles if p.PID == 321])
    # de = np.abs(energy*1e3 -1295.63)
    # if min(de) > 0.1:
        # continue
    t = tower.T
    r = np.array([tower.X,tower.Y,tower.Z])
    eta = tower.Eta
    phi = tower.Phi
    t_readout = t- np.linalg.norm(r)/c_light
    print(f'\nEta Interval: {tower.Edges[0]:1.3f}, {tower.Edges[1]:1.3f}')
    print(f'Phi Interval: {tower.Edges[2]:1.2f}, {tower.Edges[3]:1.2f}')
    # print('r = ',r,'E=',tower.P4().E(),'Eem=',tower.Eem,'Ehad=',tower.Ehad)
    print(f'E: {tower.E:1.2f}, t(s)= {t:1.2e}, t(readout)(s) = {t_readout:1.2e}, # hits = {len(tower.Particles)}')  
    print(f'Phi: {tower.Phi:1.2f}, Eta: {tower.Eta:1.2f}')
    ET = tower.E/np.cosh(tower.Eta)
    print(f'Ex: {ET*np.cos(tower.Phi):1.2f}, Ey: {ET*np.sin(tower.Phi)}')  
    continue
    for particleT in tower.Particles:
        # if particleT.PID != 321:
            # continue

        particle = None
        for p in tree.ParticleProp:
            if particleT.PID != p.PID:
                continue
            if abs(particleT.E-p.E)/particleT.E > 1e-3:
                continue
            particle = p
            break
        if particle is None:
            print(f'Particle {particleT.PID} not found!')
            continue
        r_entry = np.array([particle.X,particle.Y,particle.Z,particle.T])
        L_entry = np.linalg.norm(r_entry[:3])
        eta_entry = np.arcsinh(r_entry[2]/np.linalg.norm(r_entry[:2]))
        phi_entry = np.atan2(r_entry[1],r_entry[0])
        t_readout = r_entry[3] - np.linalg.norm(L_entry/1e3)/c_light
        print(f'   PDGID={particle.PID}, Propagation: xEntry = {r_entry[0]:1.4f}, yEntry = {r_entry[1]:1.4f}, zEntry = {r_entry[2]:1.4f}, i.e. rEntry = {L_entry:1.4f}, etaEntry = {eta_entry:1.4f}, phiEntry = {phi_entry:1.4f}, tEntry = {r_entry[3]*c_light*1e3:1.3f}, tReadoutEntry = {t_readout*c_light*1e3:1.3f}')
        p = np.array([particle.Px,particle.Py,particle.Pz,particle.E])*1e3
        print(f'Momentum px = {p[0]:1.4f}, py = {p[1]:1.4f}, pz = {p[2]:1.4f}, energy = {p[3]:1.4f}')
    break
    

#     TT 3 x 7, Calo layer: ECAL
# AnalysisAlg              INFO    Eta Interval: -3.625, -3.2
# AnalysisAlg              INFO    Phi Interval: -2.5, -2.4
# AnalysisAlg              INFO    Adding the following particles: 
# AnalysisAlg              INFO      >> PDGID 321, ProdVtx: x0 = 0, y0 = 0, z0 = 0, t0 = 0, Momentum px = -66.3621, py = -55.0955, pz = -1194.78, energy = 1295.63. Propagation: xEntry = -205.511, yEntry = -170.62, zEntry = -3700, i.e. rEntry = 3709.63, etaEntry = -3.32288, phiEntry = -2.44869, tEntry = 4012.32, tReadoutEntry = 302.687
# AnalysisAlg              INFO      >> PDGID 211, ProdVtx: x0 = 0, y0 = 0, z0 = 0, t0 = 0, Momentum px = -1212.36, py = -935.016, pz = -18760, energy = 18822.9. Propagation: xEntry = -239.11, yEntry = -184.411, zEntry = -3700, i.e. rEntry = 3712.3, etaEntry = -3.2006, phiEntry = -2.48464, tEntry = 3712.4, tReadoutEntry = 0.102051
# AnalysisAlg              INFO    TT total energy: 20118.6 (N.B. threshold for this TT is 4000)
# AnalysisAlg              INFO    Eta Interval: -3.625, -3.2, i.e. eta = -3.4125
# AnalysisAlg              INFO    Phi Interval: -2.5, -2.4, i.e. phi = -2.45
# AnalysisAlg              INFO      >> Above threshold. Adding Ex = -1020.34, Ey = -844.862

f.Close()


OnTime Calorimeter:

Eta Interval: -4.900, -4.475
Phi Interval: -2.40, -2.30
E: 5.17, t(s)= 1.23e-08, t(readout)(s) = 4.52e-12, # hits = 1
Phi: -2.35, Eta: -4.69
Ex: -0.07, Ey: -0.06772255363009298

Eta Interval: -4.900, -4.475
Phi Interval: -2.20, -2.10
E: 36.19, t(s)= 1.23e-08, t(readout)(s) = 8.93e-13, # hits = 1
Phi: -2.15, Eta: -4.69
Ex: -0.36, Ey: -0.557802893108187

Eta Interval: -4.900, -4.475
Phi Interval: -2.10, -2.00
E: 16.49, t(s)= 1.23e-08, t(readout)(s) = -6.41e-13, # hits = 1
Phi: -2.05, Eta: -4.69
Ex: -0.14, Ey: -0.2694442465217796

Eta Interval: -4.900, -4.475
Phi Interval: -1.90, -1.80
E: 28.75, t(s)= 1.23e-08, t(readout)(s) = 1.89e-12, # hits = 1
Phi: -1.85, Eta: -4.69
Ex: -0.15, Ey: -0.509093157258228

Eta Interval: -4.900, -4.475
Phi Interval: -0.60, -0.50
E: 1.26, t(s)= 1.23e-08, t(readout)(s) = 3.33e-13, # hits = 1
Phi: -0.55, Eta: -4.69
Ex: 0.02, Ey: -0.012137397872190103

Eta Interval: -4.900, -4.475
Phi Interval: -0.50, -0.40
E: 122.86, t(s)= 1.23e-08, t(read

In [177]:
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)

particle = None
check_energy = 12913.8/1e3
for p in tree.ParticleProp:
    if -321 != p.PID:
        continue
    if abs(check_energy-p.E)/check_energy > 1e-3:
        continue
    particle = p
    break
if particle is None:
    print(f'Particle not found!')
else:
    r_entry = np.array([particle.X,particle.Y,particle.Z,particle.T])
    L_entry = np.linalg.norm(r_entry[:3])
    rho_entry = np.linalg.norm(r_entry[:2])
    eta_entry = np.arcsinh(r_entry[2]/np.linalg.norm(r_entry[:2]))
    phi_entry = np.atan2(r_entry[1],r_entry[0])
    t_readout = r_entry[3] - np.linalg.norm(L_entry/1e3)/c_light
    print(f'   PDGID={particle.PID}, Propagation: xEntry = {r_entry[0]:1.4f}, yEntry = {r_entry[1]:1.4f}, zEntry = {r_entry[2]:1.4f}, i.e. rEntry = {L_entry:1.4f}, rhoEntry = {rho_entry:1.4f}, etaEntry = {eta_entry:1.4f}, phiEntry = {phi_entry:1.4f}, tEntry = {r_entry[3]*c_light*1e3:1.3f}, tReadoutEntry = {t_readout*c_light*1e3:1.3f}')
    p = np.array([particle.Px,particle.Py,particle.Pz,particle.E])*1e3
    print(f'Momentum px = {p[0]:1.4f}, py = {p[1]:1.4f}, pz = {p[2]:1.4f}, energy = {p[3]:1.4f}')
    

# TT 2 x 42, Calo layer: ECAL
# AnalysisAlg              INFO    Eta Interval: -4.05, -3.625
# AnalysisAlg              INFO    Phi Interval: 1, 1.1
# AnalysisAlg              INFO    Adding the following particles: 
# AnalysisAlg              INFO      >> PDGID -321, ProdVtx: x0 = 0, y0 = 0, z0 = 0, t0 = 0, Momentum px = 231.313, py = 413.27, pz = -12895.7, energy = 12913.8. Propagation: xEntry = 66.3678, yEntry = 118.574, zEntry = -3700, i.e. rEntry = 3702.49, etaEntry = -3.99777, phiEntry = 1.06053, tEntry = 3705.2, tReadoutEntry = 2.7085
# AnalysisAlg              INFO    TT total energy: 12913.8 (N.B. threshold for this TT is 7500)
# AnalysisAlg              INFO    Eta Interval: -4.05, -3.625, i.e. eta = -3.8375
# AnalysisAlg              INFO    Phi Interval: 1, 1.1, i.e. phi = 1.05
# AnalysisAlg              INFO      >> Above threshold. Adding Ex = 276.779, Ey = 482.513

f.Close()

   PDGID=-321, Propagation: xEntry = 66.3678, yEntry = 118.5743, zEntry = -3700.0000, i.e. rEntry = 3702.4944, rhoEntry = 135.8843, etaEntry = -3.9978, phiEntry = 1.0605, tEntry = 3705.203, tReadoutEntry = 2.709
Momentum px = 231.3130, py = 413.2697, pz = -12895.6900, energy = 12913.8241


In [178]:
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)

particle = None
check_energy = 27218./1e3
minDE = 1e10
tower_sel = None
for tower in tree.L1TowerOnTime:
    for p in tower.Particles:
        if -2212 != p.PID:
            continue
        
        if abs(check_energy-p.E)/check_energy < minDE:
            particle = p
            minDE = abs(check_energy-p.E)/check_energy
            tower_sel = tower
        
        # break
if tower_sel is None:
    # continue
    print(f'Particle not found!')
else:
    tower = tower_sel
    print(f'\nEta Interval: {tower.Edges[0]:1.3f}, {tower.Edges[1]:1.3f}')
    print(f'Phi Interval: {tower.Edges[2]:1.2f}, {tower.Edges[3]:1.2f}')
    # print('r = ',r,'E=',tower.P4().E(),'Eem=',tower.Eem,'Ehad=',tower.Ehad)
    print(f'E: {tower.E:1.2f}, t(s)= {t:1.2e}, t(readout)(s) = {t_readout:1.2e}, # hits = {len(tower.Particles)}')  
    print(f'Phi: {tower.Phi:1.2f}, Eta: {tower.Eta:1.2f}')
    ET = tower.E/np.cosh(tower.Eta)
    print(f'Ex: {ET*np.cos(tower.Phi):1.2f}, Ey: {ET*np.sin(tower.Phi)}') 
    r_entry = np.array([particle.X,particle.Y,particle.Z,particle.T])
    L_entry = np.linalg.norm(r_entry[:3])
    eta_entry = np.arcsinh(r_entry[2]/np.linalg.norm(r_entry[:2]))
    phi_entry = np.atan2(r_entry[1],r_entry[0])
    t_readout = r_entry[3] - np.linalg.norm(L_entry/1e3)/c_light
    print(f'   PDGID={particle.PID}, Propagation: xEntry = {r_entry[0]:1.4f}, yEntry = {r_entry[1]:1.4f}, zEntry = {r_entry[2]:1.4f}, i.e. rEntry = {L_entry:1.4f}, etaEntry = {eta_entry:1.4f}, phiEntry = {phi_entry:1.4f}, tEntry = {r_entry[3]*c_light*1e3:1.3f}, tReadoutEntry = {t_readout*c_light*1e3:1.3f}')
    p = np.array([particle.Px,particle.Py,particle.Pz,particle.E])*1e3
    print(f'Momentum px = {p[0]:1.4f}, py = {p[1]:1.4f}, pz = {p[2]:1.4f}, energy = {p[3]:1.4f}')


# TT 2 x 42, Calo layer: ECAL
# AnalysisAlg              INFO    Eta Interval: -4.05, -3.625
# AnalysisAlg              INFO    Phi Interval: 1, 1.1
# AnalysisAlg              INFO    Adding the following particles: 
# AnalysisAlg              INFO      >> PDGID -321, ProdVtx: x0 = 0, y0 = 0, z0 = 0, t0 = 0, Momentum px = 231.313, py = 413.27, pz = -12895.7, energy = 12913.8. Propagation: xEntry = 66.3678, yEntry = 118.574, zEntry = -3700, i.e. rEntry = 3702.49, etaEntry = -3.99777, phiEntry = 1.06053, tEntry = 3705.2, tReadoutEntry = 2.7085
# AnalysisAlg              INFO    TT total energy: 12913.8 (N.B. threshold for this TT is 7500)
# AnalysisAlg              INFO    Eta Interval: -4.05, -3.625, i.e. eta = -3.8375
# AnalysisAlg              INFO    Phi Interval: 1, 1.1, i.e. phi = 1.05
# AnalysisAlg              INFO      >> Above threshold. Adding Ex = 276.779, Ey = 482.513

f.Close()


Eta Interval: -4.475, -4.050
Phi Interval: 1.80, 1.90
E: 26.06, t(s)= 1.23e-08, t(readout)(s) = 9.03e-12, # hits = 1
Phi: 1.85, Eta: -4.26
Ex: -0.20, Ey: 0.7055477845658129
   PDGID=-2212, Propagation: xEntry = 0.0000, yEntry = 0.0000, zEntry = 0.0000, i.e. rEntry = 0.0000, etaEntry = nan, phiEntry = 0.0000, tEntry = 0.000, tReadoutEntry = 0.000
Momentum px = -195.0929, py = 638.4375, pz = -27193.6455, energy = 27218.0157


/tmp/ipykernel_44372/1299856054.py:34: RuntimeWarning: invalid value encountered in scalar divide
  eta_entry = np.arcsinh(r_entry[2]/np.linalg.norm(r_entry[:2]))


### Check energies against Tobias results

In [179]:
energyBins = {}
with open('./B2TF-Tobias/results/TT.log','r') as f:
    lines = f.readlines()

for i,l in enumerate(lines):
    l = l.strip()
    if not l: continue
    if l[:3] == 'TT ':
        nhits = 0
        ttStr,layerStr = l.split(',')
        calType = layerStr.split(':')[1].strip()
        eta = lines[i+1].split('Eta Interval:')[1].split(',')
        etaEdges = [eval(x) for x in eta]
        phi = lines[i+2].split('Phi Interval:')[1].split(',')
        phiEdges = [eval(x) for x in phi]
        egdes = tuple(etaEdges + phiEdges)
        j = i+2
        l_check = lines[j]
        while ('Adding Ex =' not in l_check):
            if 'PDGID' in l_check:
                nhits += 1
            j = j+1
            l_check = lines[j]
        if 'Below threshold' in l_check:
            Ex = 0.0
            Ey = 0.0
        else:
            ExEy = l_check.split('Adding')[1]
            Ex,Ey = ExEy.split(',')
            Ex = eval(Ex.replace(',','').split('=')[1])
            Ey = eval(Ey.split('=')[1])
        energyBins[egdes] = [Ex,Ey,nhits]


In [181]:
ExTot = 0.0
EyTot = 0.0
for Ex,Ey,nhits in energyBins.values():
    ExTot -= Ex/1e3
    EyTot -= Ey/1e3
print(ExTot,EyTot,np.sqrt(ExTot**2+EyTot**2))

-2.147144599999998 -72.12156769999994 72.15352214160431


In [182]:
energyBinsDelphes = {}
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)

for tower in tree.L1TowerOnTime:
    edges = (round(tower.Edges[0],4),round(tower.Edges[1],4),
             round(tower.Edges[2],4),round(tower.Edges[3],4))
    ET = tower.E/np.cosh(tower.Eta)
    Ex = ET*np.cos(tower.Phi)
    Ey = ET*np.sin(tower.Phi)
    nhits = len(tower.Particles)
    if edges in energyBinsDelphes:
        print('Edges already set!')
    energyBinsDelphes[edges]= [Ex,Ey,nhits]
f.Close()
    

In [183]:
ExTot = 0.0
EyTot = 0.0
for Ex,Ey,nhits in energyBinsDelphes.values():
    ExTot -= Ex
    EyTot -= Ey
print(ExTot,EyTot,np.sqrt(ExTot**2+EyTot**2))
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)
met = tree.L1METOnTime.At(0)
print(met.MET*np.cos(met.Phi),met.MET*np.sin(met.Phi),met.MET)
f.Close()

-3.2820012522703315 -90.97360432298694 91.0327864878105
-3.281997631789103 -90.97360147215285 91.03278350830078


In [192]:
print(len(energyBins),len(energyBinsDelphes))
ndiff = 0
for edges,energies_hits in energyBins.items():
    if sum(energies_hits[:2]) < 1e-2:
        continue

    # print('\n',edges)
    # print(energies_hits)
    # print(energyBinsDelphes[edges])
    

    if not edges in energyBinsDelphes:
        print(edges,energies_hits)
        ndiff +=1
    elif energies_hits[2] != energyBinsDelphes[edges][2]:
        print(edges)
        print(f'   hits (delphes) = {energies_hits[2]} ({energyBinsDelphes[edges][2]})')
    # if ndiff >= 5:
    #     break

283 221


In [188]:
print(len(energyBins),len(energyBinsDelphes))
ndiff = 0
for edges,energies_hits in energyBinsDelphes.items():
    if sum(energies_hits[:2]) < 1e-2:
        continue
    if not edges in energyBins:
        print(edges,energies_hits)
        ndiff +=1
    elif energies_hits[2] != energyBins[edges][2]:
        print(edges)
        print(f'   hits (tobias) = {energies_hits[2]} ({energyBins[edges][2]})')

    # if ndiff >= 5:
    #     break

283 221
(1.4, 1.6, 2.1, 2.2) [np.float64(-0.17830994675902237), np.float64(0.2726322424509578), 1]
